In [1]:
# 엑셀파일 읽기
import pandas as pd

In [ ]:
%pip install pandas openpyxl xlrd

In [41]:
# pd.read_excel('./lotto_1193.xls',engine='xlrd')
table = pd.read_html('./lotto_1193.xls')   # 확장자가  xls 이지만 내부에  html형태로 되어 있는 테이블
lotto_df = table[1].loc[2:, 13:]
lotto_df = lotto_df[::-1]  # 순서를 반대로
lotto_df = lotto_df.reset_index(drop=True)  # 인덱스 정렬
lotto_df.tail()

,13,14,15,16,17,18,19
588,9,19,29,35,37,38,31
589,7,9,19,23,26,45,33
590,1,4,11,12,20,41,2
591,10,16,23,36,39,40,11
592,6,9,16,19,24,28,17


In [ ]:
# 과거 2개의 데이터가 다음 차수를 예측하는 구조로 데이터 셋을 생성   x,y가 있어야 하기 때문에
# 0:2 2
# 1:3 3
# 2:4 4 
# ....
# ?:len(x)

,13,14,15,16,17,18,19
1,10,16,23,36,39,40,11
2,1,4,11,12,20,41,2


In [ ]:
# 데이터셋 생성 완료
X, y = [],[]
for i in range(len(lotto_df)-2):
    X.append(lotto_df.iloc[ i : i+2 ])
    y.append(lotto_df.iloc[i+2])  

13     6
14     9
15    16
16    19
17    24
18    28
19    17
Name: 592, dtype: object

In [38]:
lotto_df[2]

KeyError: 2

In [33]:
lotto_df.tail()

,13,14,15,16,17,18,19
588,1,2,7,9,10,38,42
589,2,6,18,21,33,34,30
590,2,19,25,26,27,43,28
591,13,14,22,27,30,38,2
592,2,16,19,31,34,35,37
